## Caso COVID-19 Fase 1
Julián Correa, Guillermo Cortés, Cristian Sarmiento

## Creditos de la información
La data utilizada en el ejercicio es tomada del **repositorio de datos COVID-19 del Centro de Ciencias de Datos e Ingeniria de Sistemas (CSSE) de la Universidad Johns Hopkins (https://github.com/CSSEGISandData/COVID-19)[https://github.com/CSSEGISandData/COVID-19]** 

### Preprocesarmiento de información

In [1]:
## import libraries and dependencies
import pandas as pd

## read downloaded csv file
covid_cases = pd.read_csv('time_series_covid19_confirmed_global.csv')
covid_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


Al realizar la inspección nos encontramos con que la tabla esta en formato horizontan donde las fechas son columnas en el data frame lo que hace complicado el procesamiento de la información. Procedemos hacer el comando **melt** de pandas para convertir **date** en variable y **value** como otra variable que contiene el numero de casos.

In [2]:
## melt dataframe to get dates
id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long']
df = covid_cases.melt(id_vars=id_vars, var_name='date')
df.sample(n=5)

,Province/State,Country/Region,Lat,Long,date,value
278650,Saskatchewan,Canada,52.939900,-106.450900,9/12/22,141815
61657,NaN,Cuba,21.521757,-77.781167,8/22/20,3617
28890,NaN,Uruguay,-32.522800,-55.765800,4/30/20,643
288347,NaN,Paraguay,-23.442500,-58.443800,10/15/22,717260
157545,Alberta,Canada,53.933300,-116.576500,7/20/21,232875


## Revisión de tipos de variables

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330327 entries, 0 to 330326
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Province/State  104013 non-null  object 
 1   Country/Region  330327 non-null  object 
 2   Lat             328041 non-null  float64
 3   Long            328041 non-null  float64
 4   date            330327 non-null  object 
 5   value           330327 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 15.1+ MB


Encontramos que la variable **date** no corresponde al tipo de valor de fecha, procedemos hacer el ajuste convirtiendo **object** a tipo **datetime64**

In [4]:
## Transform date column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%y')

df.sample(n=5)

,Province/State,Country/Region,Lat,Long,date,value
175141,NaN,Argentina,-38.4161,-63.6167,2021-09-19,5239232
101740,Queensland,Australia,-27.4698,153.0251,2021-01-08,1274
302441,NaN,Iceland,64.9631,-19.0208,2022-12-03,207101
322633,NaN,Dominican Republic,18.7357,-70.1627,2023-02-11,660412
177783,Nova Scotia,Canada,44.6820,-63.7443,2021-09-28,6598


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330327 entries, 0 to 330326
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Province/State  104013 non-null  object        
 1   Country/Region  330327 non-null  object        
 2   Lat             328041 non-null  float64       
 3   Long            328041 non-null  float64       
 4   date            330327 non-null  datetime64[ns]
 5   value           330327 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 15.1+ MB


## 1. ¿En cuál mes se presentó el mayor número de contagios?

In [6]:
# df['month_year'] = df['date'].dt.strftime('%Y-%m') ## No es eficiente en ejecución, tardda 12 segundos
df['month_year'] = df['date'].dt.to_period('M')
df_sorted = df.copy()
df_sorted.sort_values(by=['Country/Region', 'Province/State', 'date'], inplace=True)
df_sorted['new_cases'] = df.groupby(['Country/Region', 'Province/State'])['value'].diff().fillna(0)

In [7]:
df_agg_month = df_sorted.groupby(['month_year'], as_index=False)['new_cases'].sum()
df_agg_month

,month_year,new_cases
0,2020-01,9267.0
1,2020-02,69599.0
2,2020-03,20899.0
3,2020-04,53646.0
4,2020-05,37757.0
5,2020-06,18726.0
6,2020-07,29464.0
7,2020-08,31821.0
8,2020-09,48796.0
9,2020-10,91876.0


In [8]:
cond_max_cases = df_agg_month['new_cases'] == df_agg_month['new_cases'].max()
df_agg_month[cond_max_cases]

,month_year,new_cases
24,2022-01,3348699.0


In [9]:
max_month = df_agg_month[cond_max_cases]['month_year'].iloc[0]
print(f'El mes y el año con mas casos es {max_month}')

El mes y el año con mas casos es 2022-01


#### 2. ¿En ese mismo mes, cuál fue el país que reportó más contagios?

In [10]:
cond = (df_sorted['month_year'] == max_month)
df_country = df_sorted[cond].groupby(['Country/Region'], as_index=False)['new_cases'].sum()
df_country.sort_values(by='new_cases').tail()

,Country/Region,new_cases
128,Netherlands,36048.0
190,United Kingdom,41996.0
63,France,237440.0
33,Canada,849919.0
9,Australia,2154890.0


In [11]:
df_country[df_country['new_cases'] == df_country['new_cases'].max()]

,Country/Region,new_cases
9,Australia,2154890.0


#### 3. ¿Cuál es el país con el menor número de casos reportados hasta la fecha?

In [12]:
df_min_country = df.groupby('Country/Region', as_index=False).agg({'value':'sum'})
df_min_country[(df_min_country['value'] == df_min_country['value'].min())]

,Country/Region,value
93,"Korea, North",300


***
___